In [1]:
import pandas as pd

from google.cloud import bigquery
from google.cloud.bigquery import Client

In [26]:
PROJECT_ID = "bunge-demo"

START_DATE = "2021-01-01"
END_DATE = "2024-01-01"

OFFSET_START_DATE = (pd.Timestamp(START_DATE) - pd.DateOffset(days=61)).strftime("%Y-%m-%d")

In [3]:
client = bigquery.Client(PROJECT_ID)

In [4]:
query = """
SELECT 
    *
FROM
  `bigquery-public-data.iowa_liquor_sales.sales` 
LIMIT 
  5
"""
query_job = client.query(query)
df = query_job.to_dataframe()
df

I0000 00:00:1722826245.795942  901739 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,RINV-04311100003,2022-10-10,6151,UNIVERSITY LIQUORS / WDM,9250 UNIVERSITY AVE UNIT 115,WEST DES MOINES,50266,POINT(-93.834244 41.599769),None,POLK,...,948167,HENNESSY VSOP,6,1750,59.99,89.99,-18,-1619.82,-31.5,-8.32
1,RINV-04712400129,2023-06-26,3390,OKOBOJI AVENUE LIQUOR,1610 OKOBOJI AVENUE,MILFORD,51351.0,POINT(-95.149544969 43.33153601),None,DICKINSON,...,19477,MAKERS MARK,12,1000,22.50,33.75,-12,-405.00,-12.0,-3.17
2,RINV-04963200033,2023-11-17,5157,KIMMES MANSON COUNTRY STORE #10,208 MAIN ST,MANSON,50563.0,POINT(-94.535010985 42.517901986),None,CALHOUN,...,41694,UV BLUE RASPBERRY,12,1000,7.50,11.25,-12,-135.00,-12.0,-3.17
3,RINV-04536000097,2023-03-03,5916,ANOTHER ROUND / DEWITT,622 S 6TH AVE,DEWITT,52742.0,POINT(-90.53908 41.80942),None,CLINTON,...,42717,MALIBU COCONUT,12,1000,11.99,17.99,-12,-215.88,-12.0,-3.17
4,RINV-04777300067,2023-08-01,5257,MAD AVE QUIK SHOP,"405, MADISON AVE",OTTUMWA,52501.0,POINT(-92.410605275 40.999140319),None,WAPELLO,...,69637,DR MCGILLICUDDYS CHERRY,12,1000,11.00,16.50,-12,-198.00,-12.0,-3.17


In [6]:
query = f"""
WITH sales_data AS (
    SELECT 
        city,
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold,
        AVG(state_bottle_retail) AS avg_bottle_price,
        SUM(volume_sold_liters) AS total_volume_sold_liters,
        SUM(sale_dollars) AS total_sale_dollars,
        AVG(state_bottle_retail - state_bottle_cost) AS avg_bottle_profit
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE
        bottles_sold > 0
        AND sale_dollars > 0
    GROUP BY
        city, date, item_name
    HAVING
        date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
)

SELECT
    city,
    date,
    item_name,
    total_amount_sold,
    avg_bottle_price,
    total_volume_sold_liters,
    total_sale_dollars,
    avg_bottle_profit,
    -- 7-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_sale_dollars,
    -- 30-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_sale_dollars
FROM
    sales_data
ORDER BY
    city, date, item_name;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df

I0000 00:00:1722826718.633829  901739 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,city,date,item_name,total_amount_sold,avg_bottle_price,total_volume_sold_liters,total_sale_dollars,avg_bottle_profit,ma7_total_amount_sold,ma7_avg_bottle_price,ma7_total_sale_dollars,ma30_total_amount_sold,ma30_avg_bottle_price,ma30_total_sale_dollars
0,None,2023-01-23,BLACK VELVET,18,11.765,19.5,188.28,3.925,18.000000,11.765000,188.280000,18.000,11.765000,188.280
1,None,2023-01-23,BLANTONS BOURBON,6,43.130,4.5,258.78,14.380,6.000000,43.130000,258.780000,6.000,43.130000,258.780
2,None,2023-01-23,CAPTAIN MORGAN ORIGINAL SPICED,12,13.590,9.0,163.08,4.530,12.000000,13.590000,163.080000,12.000,13.590000,163.080
3,None,2023-01-23,CROWN ROYAL,24,12.740,9.0,305.76,4.250,24.000000,12.740000,305.760000,24.000,12.740000,305.760
4,None,2023-01-23,CROWN ROYAL MINI,10,14.930,0.5,149.30,4.980,10.000000,14.930000,149.300000,10.000,14.930000,149.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708740,ZWINGLE,2023-12-11,PARAMOUNT WHITE RUM,6,12.000,10.5,72.00,4.000,7.200000,11.026000,74.712000,7.200,11.026000,74.712
1708741,ZWINGLE,2023-12-11,PATRON REPOSADO,2,43.500,1.5,87.00,14.500,2.000000,43.500000,87.000000,2.000,43.500000,87.000
1708742,ZWINGLE,2023-12-11,PATRON REPOSADO 200ML,2,15.330,0.4,30.66,5.110,2.000000,15.330000,30.660000,2.000,15.330000,30.660
1708743,ZWINGLE,2023-12-11,SOUTHERN COMFORT,48,3.960,9.6,190.08,1.320,33.000000,7.647500,184.560000,33.000,7.647500,184.560


In [14]:
df[["city", "item_name"]].drop_duplicates()

,city,item_name
0,ACKLEY,99 BRAND FRUIT PUNCH PET MINI
1,ACKLEY,ABSOLUT SWEDISH VODKA 80PRF
2,ACKLEY,ADMIRAL NELSON SPICED
3,ACKLEY,BACARDI SUPERIOR
4,ACKLEY,BLACK VELVET
...,...,...
1696386,ZWINGLE,99 BLUE RASPBERRIES MINI
1696389,ZWINGLE,GREY GOOSE GLASS PACK
1696391,ZWINGLE,HENNESSY VS MINI
1696394,ZWINGLE,PATRON REPOSADO


In [7]:
query = f"""
WITH sales_data AS (
    SELECT 
        city,
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold,
        AVG(state_bottle_retail) AS avg_bottle_price,
        SUM(volume_sold_liters) AS total_volume_sold_liters,
        SUM(sale_dollars) AS total_sale_dollars,
        AVG(state_bottle_retail - state_bottle_cost) AS avg_bottle_profit
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE
        bottles_sold > 0
        AND sale_dollars > 0
        AND city IS NOT NULL
    GROUP BY
        city, date, item_name
    HAVING
        date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
),
filtered_sales AS (
    SELECT
        city,
        date,
        item_name,
        total_amount_sold,
        avg_bottle_price,
        total_volume_sold_liters,
        total_sale_dollars,
        avg_bottle_profit
    FROM
        sales_data
    WHERE
        item_name IN (
            SELECT 
                item_name
            FROM
                sales_data
            GROUP BY
                item_name
            HAVING
                SUM(total_amount_sold) > 100  -- Adjust the threshold as needed
        )
)
SELECT
    city,
    date,
    item_name,
    total_amount_sold,
    avg_bottle_price,
    total_volume_sold_liters,
    total_sale_dollars,
    avg_bottle_profit,
    -- 7-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_sale_dollars,
    -- 30-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_sale_dollars,
    -- 60-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_total_sale_dollars
FROM
    filtered_sales
ORDER BY
    date, city, item_name;
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df

I0000 00:00:1722826969.196311  901739 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,city,date,item_name,total_amount_sold,avg_bottle_price,total_volume_sold_liters,total_sale_dollars,avg_bottle_profit,ma7_total_amount_sold,ma7_avg_bottle_price,ma7_total_sale_dollars,ma30_total_amount_sold,ma30_avg_bottle_price,ma30_total_sale_dollars
0,ACKLEY,2023-01-04,99 BRAND FRUIT PUNCH PET MINI,1,77.40,0.05,77.40,25.80,1.000000,77.400000,77.400000,1.000,77.400000,77.400
1,ACKLEY,2023-01-04,ABSOLUT SWEDISH VODKA 80PRF,8,14.99,6.00,119.92,5.00,8.000000,14.990000,119.920000,8.000,14.990000,119.920
2,ACKLEY,2023-01-04,ADMIRAL NELSON SPICED,4,4.49,1.50,17.96,1.50,4.000000,4.490000,17.960000,4.000,4.490000,17.960
3,ACKLEY,2023-01-04,BACARDI SUPERIOR,4,7.50,1.50,30.00,2.50,4.000000,7.500000,30.000000,4.000,7.500000,30.000
4,ACKLEY,2023-01-04,BLACK VELVET,20,6.23,12.00,131.08,2.08,20.000000,6.230000,131.080000,20.000,6.230000,131.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1696393,ZWINGLE,2023-12-11,PARAMOUNT WHITE RUM,6,12.00,10.50,72.00,4.00,7.200000,11.026000,74.712000,7.200,11.026000,74.712
1696394,ZWINGLE,2023-12-11,PATRON REPOSADO,2,43.50,1.50,87.00,14.50,2.000000,43.500000,87.000000,2.000,43.500000,87.000
1696395,ZWINGLE,2023-12-11,PATRON REPOSADO 200ML,2,15.33,0.40,30.66,5.11,2.000000,15.330000,30.660000,2.000,15.330000,30.660
1696396,ZWINGLE,2023-12-11,SOUTHERN COMFORT,48,3.96,9.60,190.08,1.32,33.000000,7.647500,184.560000,33.000,7.647500,184.560


In [51]:
START_DATE = "2021-01-01"
END_DATE = "2024-02-01"

OFFSET_START_DATE = (pd.Timestamp(START_DATE) - pd.DateOffset(days=61)).strftime("%Y-%m-%d")

query = f"""
WITH sales_data AS (
    SELECT 
        city,
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold,
        AVG(state_bottle_retail) AS avg_bottle_price,
        SUM(volume_sold_liters) AS total_volume_sold_liters,
        SUM(sale_dollars) AS total_sale_dollars,
        AVG(state_bottle_retail - state_bottle_cost) AS avg_bottle_profit
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE
        bottles_sold > 0
        AND sale_dollars > 0
        AND city IS NOT NULL
    GROUP BY
        city, date, item_name
    HAVING
        date >= DATE('{OFFSET_START_DATE}')
),
filtered_sales AS (
    SELECT
        sd.city,
        sd.date,
        sd.item_name,
        sd.total_amount_sold,
        sd.avg_bottle_price,
        sd.total_volume_sold_liters,
        sd.total_sale_dollars,
        sd.avg_bottle_profit
    FROM
        sales_data AS sd
    INNER JOIN
        (
            SELECT
                d2.city,
                d2.item_name
            FROM
                (
                    SELECT
                        sd.city,
                        sd.item_name,
                        EXTRACT(YEAR FROM date) AS year
                    FROM
                        sales_data AS sd
                    WHERE
                        sd.date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
                    GROUP BY
                        year, sd.city, sd.item_name 
                    HAVING
                        SUM(sd.total_amount_sold) > 100  -- Adjust the threshold as needed
                        AND COUNT(DISTINCT sd.date) > 180
                ) AS d2
            GROUP BY
                d2.city, d2.item_name
            HAVING
                COUNT(DISTINCT d2.year) >= 3
                --AND MAX(d2.year) = 2024
        ) AS d3
    ON
        sd.city = d3.city
        AND sd.item_name = d3.item_name
    INNER JOIN
        (
            SELECT
                d4.city,
                d4.item_name,
                MAX(EXTRACT(YEAR FROM date)) AS max_year
            FROM
                sales_data AS d4
            GROUP BY
                d4.city, d4.item_name 
            HAVING
                max_year = 2024
        ) AS d5
    ON
        sd.city = d5.city
        AND sd.item_name = d5.item_name
)
SELECT
    city,
    date,
    item_name,
    total_amount_sold,
    avg_bottle_price,
    total_volume_sold_liters,
    total_sale_dollars,
    avg_bottle_profit,
    -- 7-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS ma7_total_sale_dollars,
    -- 30-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS ma30_total_sale_dollars,
    -- 60-day moving averages
    AVG(total_amount_sold) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_total_amount_sold,
    AVG(avg_bottle_price) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_avg_bottle_price,
    AVG(total_sale_dollars) OVER (PARTITION BY city, item_name ORDER BY date ROWS BETWEEN 59 PRECEDING AND CURRENT ROW) AS ma60_total_sale_dollars
FROM
    filtered_sales
WHERE
    date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
ORDER BY
    city, item_name, date;
"""
query_job = client.query(query)
df = query_job.to_dataframe()
df

I0000 00:00:1722836851.011240  901739 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,city,date,item_name,total_amount_sold,avg_bottle_price,total_volume_sold_liters,total_sale_dollars,avg_bottle_profit,ma7_total_amount_sold,ma7_avg_bottle_price,ma7_total_sale_dollars,ma30_total_amount_sold,ma30_avg_bottle_price,ma30_total_sale_dollars,ma60_total_amount_sold,ma60_avg_bottle_price,ma60_total_sale_dollars
0,CEDAR RAPIDS,2021-01-04,BACARDI SUPERIOR,82,10.980000,48.00,812.94,3.660000,82.000000,10.980000,812.940000,82.000000,10.980000,812.940000,82.000000,10.980000,812.940000
1,CEDAR RAPIDS,2021-01-06,BACARDI SUPERIOR,43,10.035000,35.62,537.93,3.345000,62.500000,10.507500,675.435000,62.500000,10.507500,675.435000,62.500000,10.507500,675.435000
2,CEDAR RAPIDS,2021-01-07,BACARDI SUPERIOR,52,12.815000,47.25,693.18,4.271667,59.000000,11.276667,681.350000,59.000000,11.276667,681.350000,59.000000,11.276667,681.350000
3,CEDAR RAPIDS,2021-01-11,BACARDI SUPERIOR,55,11.066250,42.75,651.63,3.688750,58.000000,11.224063,673.920000,58.000000,11.224063,673.920000,58.000000,11.224063,673.920000
4,CEDAR RAPIDS,2021-01-13,BACARDI SUPERIOR,61,10.625000,38.62,596.25,3.541667,58.600000,11.104250,658.386000,58.600000,11.104250,658.386000,58.600000,11.104250,658.386000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42923,WEST DES MOINES,2024-01-26,TITOS HANDMADE VODKA,37,19.148333,35.75,654.08,6.383333,125.428571,17.743169,2578.807143,184.966667,17.960122,3436.731333,217.216667,17.722297,4205.232167
42924,WEST DES MOINES,2024-01-29,TITOS HANDMADE VODKA,441,18.088276,423.05,7751.42,6.030000,169.000000,17.838955,3278.490000,180.200000,18.027031,3428.913667,224.366667,17.694435,4330.470500
42925,WEST DES MOINES,2024-01-30,TITOS HANDMADE VODKA,42,21.750000,37.50,711.00,7.250000,172.428571,19.130383,3343.452857,172.000000,18.027031,3200.613667,224.666667,17.864185,4337.694500
42926,WEST DES MOINES,2024-01-31,TITOS HANDMADE VODKA,200,14.578750,161.30,3092.60,4.860000,167.857143,18.802004,3249.274286,173.466667,17.958739,3226.804333,214.766667,17.842416,4117.912833


In [52]:
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df.groupby(["city", "item_name"], as_index=True).agg({"year": ["unique", "nunique", "max", "min"]}).sort_values(("year", "nunique"))

year  \
                                                                       unique   
city            item_name                                                       
CEDAR RAPIDS    BACARDI SUPERIOR                     [2021, 2022, 2023, 2024]   
DES MOINES      GREY GOOSE                           [2021, 2022, 2023, 2024]   
                HAWKEYE VODKA                        [2021, 2022, 2023, 2024]   
                JACK DANIELS OLD #7 BLACK LABEL      [2021, 2022, 2023, 2024]   
                JAGERMEISTER LIQUEUR                 [2021, 2022, 2023, 2024]   
...                                                                       ...   
                CAPTAIN MORGAN ORIGINAL SPICED MINI  [2021, 2022, 2023, 2024]   
                CHRISTIAN BROS BRANDY                [2021, 2022, 2023, 2024]   
                CROWN ROYAL                          [2021, 2022, 2023, 2024]   
                E & J VS                             [2021, 2022, 2023, 2024]   
WEST DES MOINES TITOS HANDMADE VODKA                 [2021, 2022, 2023, 2024]   

                                                                         
                                                    nunique   max   min  
city            item_name                                                
CEDAR RAPIDS    BACARDI SUPERIOR                          4  2024  2021  
DES MOINES      GREY GOOSE                                4  2024  2021  
                HAWKEYE VODKA                             4  2024  2021  
                JACK DANIELS OLD #7 BLACK LABEL           4  2024  2021  
                JAGERMEISTER LIQUEUR                      4  2024  2021  
...                                                     ...   ...   ...  
                CAPTAIN MORGAN ORIGINAL SPICED MINI       4  2024  2021  
                CHRISTIAN BROS BRANDY                     4  2024  2021  
                CROWN ROYAL                               4  2024  2021  
                E & J VS                                  4  2024  2021  
WEST DES MOINES TITOS HANDMADE VODKA                      4  2024  2021  

[65 rows x 4 columns]

In [9]:
print(*df["item_name"].sort_values().unique(), sep="\n")

10TH MOUNTAIN BOURBON
135 EAST HYOGO JAPANESE DRY GIN
173 CRAFT DISTILLERY BARREL & KANE
173 CRAFT DISTILLERY BOURBON WHISKEY
173 CRAFT DISTILLERY BROKEN BEAKER SILVER RUM
173 CRAFT DISTILLERY PREMIUM GIN
173 CRAFT DISTILLERY PREMIUM VODKA
173 CRAFT DISTILLERY SPICED RUM
173 CRAFT DISTILLERY VOLUMETRIC GIN
173 CRAFT DISTILLERY VOLUMETRIC VODKA
1792 BOTTLED IN BOND BOURBON
1792 BOTTLED IN BOND BOURBON BARREL
1792 CHOCOLATE BOURBON BALL CREAM
1792 FULL PROOF BUY THE BARREL
1792 SINGLE BARREL BUY THE BARREL
1792 SMALL BATCH
1792 SMALL BATCH BOURBON
1792 SWEET WHEAT BOURBON
1800 ANEJO
1800 COCONUT
1800 CRISTALINO
1800 CUCUMBER & JALAPENO
1800 PASSION FRUIT ULTIMATE MARGARITA
1800 REPOSADO
1800 REPOSADO W/CLAY CUP
1800 SILVER
1800 SILVER MINI
1800 SILVER W/TACO HOLDER
1800 ULTIMATE BLACK CHERRY MARGARITA PET
1800 ULTIMATE BLOOD ORANGE MARGARITA PET
1800 ULTIMATE MANGO MARGARITA PET
1800 ULTIMATE MARGARITA PET
1800 ULTIMATE PEACH MARGARITA PET
1800 ULTIMATE PINEAPPLE MARGARITA PET
1800 ULTIM

In [8]:
df["city"].unique()

array(['ACKLEY', 'ADAIR', 'ADEL', 'AFTON', 'AINSWORTH', 'AKRON',
       'ALBERT CITY', 'ALBIA', 'ALBION', 'ALBURNETT', 'ALDEN', 'ALGONA',
       'ALLERTON', 'ALLISON', 'ALTA', 'ALTON', 'ALTOONA', 'AMANA', 'AMES',
       'ANAMOSA', 'ANITA', 'ANKENY', 'ANTHON', 'APLINGTON', 'ARLINGTON',
       'ARMSTRONG', 'ARNOLDS PARK', 'ATKINS', 'ATLANTIC', 'AUDUBON',
       'AURELIA', 'AVOCA', 'BALDWIN', 'BANCROFT', 'BATAVIA', 'BAXTER',
       'BEDFORD', 'BELLE PLAINE', 'BELLEVUE', 'BELMOND', 'BETTENDORF',
       'BEVINGTON', 'BIRMINGHAM', 'BLAIRSTOWN', 'BLOOMFIELD',
       'BLUE GRASS', 'BONAPARTE', 'BONDURANT', 'BOONE', 'BOYDEN', 'BRITT',
       'BROOKLYN', 'BUFFALO', 'BUFFALO CENTER', 'BURLINGTON', 'CALAMUS',
       'CALMAR', 'CAMANCHE', 'CARLISLE', 'CARROLL', 'CARSON',
       'CARTER LAKE', 'CASCADE', 'CASEY', 'CEDAR FALLS', 'CEDAR RAPIDS',
       'CENTER POINT', 'CENTERVILLE', 'CENTRAL CITY', 'CHARITON',
       'CHARLES CITY', 'CHEROKEE', 'CLARENCE', 'CLARINDA', 'CLARION',
       'CLARKSVILLE', 

In [5]:
old_training_data_query = f"""
SELECT 
    city,
    date,
    item_description AS item_name,
    SUM(bottles_sold) AS total_amount_sold,
    AVG(state_bottle_retail) AS avg_bottle_price,
    SUM(volume_sold_liters) AS total_volume_sold_liters,
    SUM(sale_dollars) AS total_sale_dollars,
    AVG(state_bottle_retail - state_bottle_cost) AS avg_bottle_profit
FROM
    `bigquery-public-data.iowa_liquor_sales.sales`
WHERE
    bottles_sold > 0
    AND sale_dollars > 0
GROUP BY
    city, date, item_name
HAVING
    date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
"""
old_training_data_query_job = client.query(old_training_data_query)
old_df = old_training_data_query_job.to_dataframe()
old_df

I0000 00:00:1722826458.926348  901739 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,city,date,item_name,total_amount_sold,avg_bottle_price,total_volume_sold_liters,total_sale_dollars,avg_bottle_profit
0,WATERLOO,2023-03-09,FIREBALL CINNAMON WHISKEY PARTY BUCKET,3,77.40,0.15,232.20,25.80
1,ANKENY,2023-05-24,NEW AMSTERDAM PINK WHITNEY MINI,13,6.92,0.65,89.96,2.31
2,URBANDALE,2023-01-12,FIREBALL CINNAMON WHISKEY MINI DISPENSER,9,38.70,0.45,348.30,12.90
3,FORT DODGE,2023-08-04,TEQUILA ROSE LIQUEUR MINI,7,7.05,0.35,49.35,2.35
4,SIOUX CITY,2023-12-12,99 BLUE RASPBERRIES MINI,5,77.40,0.25,387.00,25.80
...,...,...,...,...,...,...,...,...
1708740,DES MOINES,2023-02-16,REMY MARTIN VSOP,2,11.82,0.40,23.64,3.94
1708741,CEDAR RAPIDS,2023-09-11,OLE SMOKY WHITE CHOCOLATE STRAWBERRY MOONSHINE...,8,13.13,0.40,105.04,4.38
1708742,HIAWATHA,2023-04-14,BLACK VELVET MINI,8,13.31,0.40,106.48,4.44
1708743,DAVENPORT,2023-03-20,OLE SMOKY BUTTER PECAN MOONSHINE MINI,8,13.13,0.40,105.04,4.38


In [27]:
training_data_query = f"""
WITH sales_per_day AS (
    SELECT
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE
        date BETWEEN DATE('{START_DATE}') AND DATE('{END_DATE}')
        AND bottles_sold > 0
    GROUP BY
        date, item_name
),
sales_per_month AS (
    SELECT
        item_name,
        EXTRACT(YEAR FROM date) AS year,
        EXTRACT(MONTH FROM date) AS month,
        COUNT(DISTINCT date) AS days_with_sales
    FROM
        sales_per_day
    GROUP BY
        item_name, year, month
),
qualified_products AS (
    SELECT
        item_name
    FROM
        sales_per_month
    WHERE
        days_with_sales >= 10
    GROUP BY
        item_name, year, month
    HAVING
        COUNT(DISTINCT CONCAT(year, '-', month)) = COUNT(*)
)
SELECT
    date,
    item_name,
    total_amount_sold
FROM
    sales_per_day
WHERE
    item_name IN (SELECT item_name FROM qualified_products)
"""
training_data_query_job = client.query(training_data_query)
df = training_data_query_job.to_dataframe()
df

I0000 00:00:1722632878.925677  412845 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,date,item_name,total_amount_sold
0,2023-06-30,FIREBALL CINNAMON WHISKEY PARTY BUCKET,107
1,2023-05-30,99 FRUITS MINI,76
2,2023-05-23,CROWN ROYAL REGAL APPLE MINI,207
3,2023-06-30,CROWN ROYAL MINI,153
4,2023-03-30,BLACK VELVET MINI,89
...,...,...,...
345921,2023-10-30,BURNETTS RASPBERRY,60
345922,2023-02-07,MR BOSTON LIGHT RUM,60
345923,2023-01-03,DON AGAVE GOLD,60
345924,2023-08-28,ON THE ROCKS COCKTAILS LARIOS AVIATION,60


In [46]:
qty_of_months = len(pd.date_range(start=pd.to_datetime(START_DATE), end=pd.to_datetime(END_DATE), freq="M"))

In [64]:
training_data_query = f"""
WITH sales_per_day AS (
    SELECT
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales`
    WHERE
        date BETWEEN DATE('2023-01-01') AND DATE('2024-01-01')
    GROUP BY
        date, item_name
),
sales_per_month AS (
    SELECT
        item_name,
        EXTRACT(YEAR FROM date) AS year,
        EXTRACT(MONTH FROM date) AS month,
        COUNT(DISTINCT date) AS days_with_sales
    FROM
        sales_per_day
    GROUP BY
        item_name, year, month
),
qualified_products AS (
    SELECT
        item_name
    FROM
        sales_per_month
    WHERE
        days_with_sales >= 10
    GROUP BY
        item_name
    HAVING
        COUNT(DISTINCT CONCAT(year, '-', month)) = (
            SELECT COUNT(DISTINCT EXTRACT(YEAR FROM date) || '-' || EXTRACT(MONTH FROM date))
            FROM sales_per_day
        )
)
SELECT
    spd.date,
    spd.item_name,
    SUM(spd.total_amount_sold) AS total_amount_sold
FROM
    sales_per_day spd
JOIN
    qualified_products qp ON spd.item_name = qp.item_name
GROUP BY
    spd.date, spd.item_name
"""
training_data_query_job = client.query(training_data_query)
df = training_data_query_job.to_dataframe()
df

I0000 00:00:1722634533.588629  412845 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


,date,item_name,total_amount_sold


In [60]:
df.loc[df["item_name"] == "WESTWARD CASK STRENGTH 125 WHISKEY"]

,date,item_name,total_amount_sold
93855,2023-06-29,WESTWARD CASK STRENGTH 125 WHISKEY,1
94046,2023-10-02,WESTWARD CASK STRENGTH 125 WHISKEY,1
96655,2023-06-03,WESTWARD CASK STRENGTH 125 WHISKEY,1
97246,2023-06-16,WESTWARD CASK STRENGTH 125 WHISKEY,1
97426,2023-10-09,WESTWARD CASK STRENGTH 125 WHISKEY,1
97978,2023-06-23,WESTWARD CASK STRENGTH 125 WHISKEY,1
99798,2023-09-01,WESTWARD CASK STRENGTH 125 WHISKEY,1
100259,2023-11-22,WESTWARD CASK STRENGTH 125 WHISKEY,1
100351,2023-08-07,WESTWARD CASK STRENGTH 125 WHISKEY,1
100362,2023-07-07,WESTWARD CASK STRENGTH 125 WHISKEY,1


In [61]:
# item_df = df.loc[(df["item_name"] == "WESTWARD CASK STRENGTH 125 WHISKEY	") & (df["total_amount_sold"] > 0)]
# item_df = df.loc[df["total_amount_sold"] > 0]
item_df = df.copy()

item_df["date"] = pd.to_datetime(item_df["date"])

item_df["year"] = item_df["date"].dt.year
item_df["month"] = item_df["date"].dt.month

item_unique_monthly_days = item_df.groupby(["item_name", "year", "month"], as_index=False)["total_amount_sold"].nunique()
item_unique_monthly_days

,item_name,year,month,total_amount_sold
0,10TH MOUNTAIN BOURBON,2023,12,11
1,173 CRAFT DISTILLERY BARREL & KANE,2023,11,11
2,173 CRAFT DISTILLERY BARREL & KANE,2023,12,6
3,173 CRAFT DISTILLERY PREMIUM GIN,2023,11,9
4,173 CRAFT DISTILLERY PREMIUM GIN,2023,12,4
...,...,...,...,...
22416,ZING ZANG BLOODY MARY RTS,2023,8,11
22417,ZING ZANG BLOODY MARY RTS,2023,9,11
22418,ZING ZANG BLOODY MARY RTS,2023,10,8
22419,ZING ZANG BLOODY MARY RTS,2023,11,10


In [62]:
df.loc[df["item_name"] == "WESTWARD CASK STRENGTH 125 WHISKEY"]

,date,item_name,total_amount_sold
93855,2023-06-29,WESTWARD CASK STRENGTH 125 WHISKEY,1
94046,2023-10-02,WESTWARD CASK STRENGTH 125 WHISKEY,1
96655,2023-06-03,WESTWARD CASK STRENGTH 125 WHISKEY,1
97246,2023-06-16,WESTWARD CASK STRENGTH 125 WHISKEY,1
97426,2023-10-09,WESTWARD CASK STRENGTH 125 WHISKEY,1
97978,2023-06-23,WESTWARD CASK STRENGTH 125 WHISKEY,1
99798,2023-09-01,WESTWARD CASK STRENGTH 125 WHISKEY,1
100259,2023-11-22,WESTWARD CASK STRENGTH 125 WHISKEY,1
100351,2023-08-07,WESTWARD CASK STRENGTH 125 WHISKEY,1
100362,2023-07-07,WESTWARD CASK STRENGTH 125 WHISKEY,1


In [63]:
item_unique_monthly_days.sort_values("total_amount_sold")

,item_name,year,month,total_amount_sold
21771,WESTWARD CASK STRENGTH 125 WHISKEY,2023,6,1
17724,SHANKYS WHIP BLACK SPIRITS MINI,2023,1,1
10836,JIM BEAM MINI,2024,1,1
17600,SEAGRAMS EXTRA DRY GIN MINI,2023,3,1
6466,DEKUYPER BUTTERSHOTS,2024,1,1
...,...,...,...,...
9552,HENNESSY VS,2023,11,29
395,99 BANANAS,2023,11,29
8267,FIREBALL CINNAMON WHISKEY MINI SLEEVE,2023,11,29
12926,MCCORMICK 80PRF VODKA,2023,11,29


In [13]:
old_unique_item_counts = old_df["item_name"].nunique()
unique_item_counts = df["item_name"].nunique()

print(
    f"Previous dataset unique items count: \033[1;31m{old_unique_item_counts:,}\033[0m | New dataset unique items count: \033[1;32m{unique_item_counts:,}\033[0m",
    f"Difference: \033[1;31m{old_unique_item_counts - unique_item_counts:,}\033[0m",
    f"Percentage reduction: \033[1;31m{(old_unique_item_counts - unique_item_counts)/old_unique_item_counts:.2%}\033[0m",
    sep="\n",
)

Previous dataset unique items count: 4,552 | New dataset unique items count: 2,276
Difference: 2,276
Percentage reduction: 50.00%


In [3]:
training_data_table_creation_query = """
CREATE OR REPLACE VIEW bqmlforecast.training_data AS
(
    SELECT 
        date,
        item_description AS item_name,
        SUM(bottles_sold) AS total_amount_sold
    FROM
        `bigquery-public-data.iowa_liquor_sales.sales` 
    GROUP BY
        date, item_name
    HAVING 
        date BETWEEN DATE('2023-01-01') AND DATE('2024-01-01')
)
"""
training_data_table_creation_job = client.query(training_data_table_creation_query)

In [4]:
create_model_query = """
CREATE OR REPLACE MODEL bqmlforecast.arima_model

OPTIONS(
  MODEL_TYPE='ARIMA',
  TIME_SERIES_TIMESTAMP_COL='date', 
  TIME_SERIES_DATA_COL='total_amount_sold',
  TIME_SERIES_ID_COL='item_name',
  HOLIDAY_REGION='US'
) AS

SELECT 
    date,
    item_name,
    total_amount_sold
FROM
  bqmlforecast.training_data
"""
create_model_job = client.query(create_model_query)

In [ ]:
evaluate_query = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL bqmlforecast.arima_model)
"""
